In [5]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
caminhos =[
    "../arquivos/Explorando o Universo das IAs com Hugging Face.pdf",
    "../arquivos/Apostila LangChain.pdf",
    "../arquivos/Explorando a API da OpenAI.pdf",
    
]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())
    
recur_split= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)

documents = recur_split.split_documents(paginas)

for i, doc in enumerate(documents):
    doc.metadata["source"] = doc.metadata["source"].replace("../arquivos/", "")
    doc.metadata["doc_id"] = i

In [7]:
diretorio = '../arquivos/chat_retrieval_db'

embedding_model = OpenAIEmbeddings()


vectordb = Chroma.from_documents(
    documents = documents,
    embedding=embedding_model,
    persist_directory=diretorio
)

## Criando Estrutura de Chat


In [8]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",)



In [9]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    
    )

In [10]:
pergunta = "Oque é Hugging Face e como faço para usar?"

chat_chain.invoke({"query": pergunta})

{'query': 'Oque é Hugging Face e como faço para usar?',
 'result': 'O Hugging Face é uma plataforma que oferece uma variedade de modelos de inteligência artificial pré-treinados, datasets e outras ferramentas relacionadas a IA. Para usar o Hugging Face, é necessário criar uma conta na plataforma, seguir os passos de autenticação e autorização para acessar modelos restritos, e então é possível utilizar os modelos disponíveis para diferentes tarefas de processamento de linguagem natural e outras aplicações de IA.'}

#### Modificando Prompt da chain 

In [11]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(
    """
    Utilize o contexto fornecido para responder a pergunta ao final .
    Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.
    Utilize três frases no máximo e mantenha a resposta sucinta.
    
    Contexto: {context}
    
    Pergunta: {question}
    
    Resposta:
    
    """
)

In [12]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type_kwargs={"prompt": chain_prompt},
    return_source_documents=True,
    
    )



In [13]:
pergunta = "Oque é Hugging Face e como faço para usar?"

chat_chain.invoke({"query": pergunta})

{'query': 'Oque é Hugging Face e como faço para usar?',
 'result': 'Hugging Face é uma plataforma de IA aberta. Para usar, é necessário criar uma conta e autenticar com um token de acesso.',
 'source_documents': [Document(metadata={'doc_id': 1, 'page': 1, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}, page_content='Explorando o Universo das IAs com Hugging Face\nConteúdo\n01. O que é Hugging Face? 5\nAfinal, o que é Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\nÉ tudo aberto mesmo? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\nComo usaremos o Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\n02. A plataforma Hugging Face 7\nModelos . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7\nPágina de um modelo . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 8\nDatasets . . . . . . . . . . . . . . . . . . . . . . . . 

### Stuff

In [14]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='stuff',
    
    )

pergunta = "Oque é Hugging Face e como faço para usar?"

resposta = chat_chain.invoke({"query": pergunta})
print(resposta["result"])


Hugging Face é uma plataforma que oferece uma variedade de modelos de inteligência artificial pré-treinados, datasets e espaços para colaboração na área de IA. Para usar o Hugging Face, você precisa criar uma conta na plataforma, seguir os passos de autenticação e, se necessário, solicitar autorização de uso de modelos restritos. Depois de criar sua conta, você pode explorar os modelos disponíveis, utilizar códigos para baixar e executar os modelos localmente, passar argumentos e obter resultados. A plataforma oferece a possibilidade de combinar diferentes modelos do Hugging Face em suas aplicações, assim como integrá-los com outras ferramentas de IA mais conhecidas.


### Map Reduce

In [15]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='map_reduce',
    
    )

pergunta = "Oque é Hugging Face e como faço para usar?"

resposta = chat_chain.invoke({"query": pergunta})
print(resposta["result"])

O Hugging Face é uma plataforma de inteligência artificial que oferece uma variedade de recursos, como modelos de IA pré-treinados, ferramentas de treinamento de modelos personalizados e uma comunidade ativa de desenvolvedores e entusiastas de IA. Para usar o Hugging Face, você pode acessar o site oficial da plataforma em huggingface.co, criar uma conta, explorar e escolher os modelos disponíveis que melhor se adequam ao seu projeto. Além disso, você pode participar da comunidade, seguir perfis e organizações, acessar recursos como artigos de blog, cursos, Discord, fórum e GitHub oficial.


### Refine

In [16]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='refine',
    
    )

pergunta = "Oque é Hugging Face e como faço para usar?"

resposta = chat_chain.invoke({"query": pergunta})
print(resposta["result"])

Hugging Face é uma plataforma que fornece acesso a modelos de Inteligência Artificial (IA), conjuntos de dados e outras ferramentas relacionadas à IA, com uma forte ênfase na comunidade. Para usar modelos restritos no Hugging Face, é necessário criar uma conta na plataforma e solicitar autorização para acessar o modelo desejado. Além disso, é preciso gerar um token de acesso para identificar o usuário que está tentando usar o modelo e conceder acesso a ele. 

O Hugging Face oferece uma experiência colaborativa e interativa para explorar o universo das IA e interagir com uma comunidade ativa de desenvolvedores e entusiastas da inteligência artificial. Ao utilizar o Hugging Face, os usuários podem combinar funcionalidades de diversos modelos para suas aplicações, integrando-os com outros scripts ou programas. Isso permite criar sistemas robustos com diversas IAs para uma variedade de tarefas, ampliando as possibilidades de uso da plataforma.
